In [ ]:
%matplotlib inline

import pickle

import dgl
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import torch
import yaml

from src.model.get_model import get_model
from src.utils.preprocess_data import preprocess_data
from src.utils.get_target import generate_target_trajectory

matplotlib.rcParams['font.family'] = ['Noto Serif', 'Serif']
plt.style.use('seaborn-poster')
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Figure 1. Prediction Performance

In [ ]:
model_names = ['Linear', 'GNN', 'ICGNN']

# Load Data
data_generation_config = yaml.safe_load(open('config/data/data_generation_config.yaml', 'r'))
data_preprocessing_config = yaml.safe_load(open('config/data/data_preprocessing_config.yaml', 'r'))
test_data = pickle.load(open(data_generation_config['test_data_saved_path'], 'rb'))
test_config = yaml.safe_load(open('config/prediction/test_config.yaml', 'r'))

# Do Prediction Task
history_len = test_config['history_len']
receding_horizon = test_config['receding_horizon']

data_preprocessing_config['history_len'] = history_len
data_preprocessing_config['receding_horizon'] = receding_horizon
data_preprocessing_config['device'] = device
test_hist_xs, test_hist_us, test_future_us, test_future_xs, test_gs, test_idxs = preprocess_data(test_data,
                                                                                                 data_preprocessing_config)
with torch.no_grad():
    tg = dgl.batch([test_gs[idx[0]] for idx in test_idxs])
    thx = torch.cat([test_hist_xs[idx[0]][idx[1]] for idx in test_idxs])
    thu = torch.cat([test_hist_us[idx[0]][idx[1]] for idx in test_idxs])
    tfu = torch.cat([test_future_us[idx[0]][idx[1]] for idx in test_idxs])
    tfx = torch.cat([test_future_xs[idx[0]][idx[1]] for idx in test_idxs])

model_loss_dict = {}

def crit(x, y):
    loss_fn = torch.nn.SmoothL1Loss(reduction='none')
    mean = loss_fn(x, y).mean(dim=(0, 2)).detach().cpu().numpy()
    std = loss_fn(x, y).std(dim=(0, 2)).detach().cpu().numpy()
    mean = np.concatenate([[0], mean])
    std = np.concatenate([[0], std])
    return mean, std

for model_name in model_names:
    model_config = yaml.safe_load(open('config/model/{}/model_config.yaml'.format(model_name), 'r'))
    m = get_model(model_name, model_config, True).to(device)
    with torch.no_grad():
        pfx = m.multistep_prediction(tg, thx, thu, tfu)
        model_loss_dict[model_name] = crit(tfx, pfx)

# Visualize Prediction Performance

gamma = 0.1
label_size = 15
fig, axes = plt.subplots(1, 1, figsize=(7, 4))
for model_name in model_loss_dict.keys():
    mean = model_loss_dict[model_name][0]
    std = model_loss_dict[model_name][1]
    axes.plot(mean, label=model_name, linewidth=2)
    axes.fill_between(range(mean.shape[0]),
                      mean - gamma * std,
                      mean + gamma * std, alpha=0.2)
axes.set_xlabel(r'Rollout Steps', fontsize=label_size)
axes.set_ylabel(r'Prediction MSE', fontsize=label_size)
axes.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
axes.legend(fancybox=True, shadow=True)
axes.grid(True, which='both', ls='--')
axes.set_xlim(0, 10)
fig.savefig('prediction.pdf', bbox_inches='tight')
plt.show()

# Figure 2. Heater Location Optimization Performance Loss vs # of Optimization Steps

In [ ]:
target = pickle.load(open('data/bilevel_design_opt/target.pkl', 'rb'))

color_dict = {
    'Linear': u'#1f77b4',
    'GAT': u'#ff7f0e',
    'ICGAT': u'#2ca02c'
}

def measure_design_opt_control(mpc_config, x_trajectory_list, u_trajectory_list):
    ridge_coefficient = mpc_config['ridge_coefficient']
    smoothness_coefficient = mpc_config['smoothness_coefficient']
    target_values_list = mpc_config['target_values_list']
    target_times_list = mpc_config['target_times_list']

    num_sensors = x_trajectory_list[0].shape[1]
    num_heaters = u_trajectory_list[0].shape[1]
    target_list = []
    for (target_values, target_times) in zip(target_values_list, target_times_list):
        target = np.array(generate_target_trajectory(target_values, target_times))
        target = np.reshape(target, newshape=(-1, 1))
        target = np.concatenate([target for _ in range(num_sensors)], axis=1)
        target_list.append(target)
    loss_list = []

    def compute_mpc_loss(x_traj, u_traj, target):
        loss_obj = np.square(x_traj[1:] - target).mean(axis=0).sum()
        loss_ridge = ridge_coefficient * np.square(u_traj).mean(axis=0).sum()
        u_prev = np.concatenate([np.zeros((1, num_heaters)), u_traj[:-1]], axis=0)
        loss_smoothness = smoothness_coefficient * np.square(u_traj - u_prev).mean(axis=0).sum()
        return loss_obj + loss_ridge + loss_smoothness

    for (x_traj, u_traj, target) in zip(x_trajectory_list, u_trajectory_list, target_list):
        loss_list.append(compute_mpc_loss(x_traj, u_traj, target))
    return np.array(loss_list)

num_x = 3
num_heaters = 5
model_names = ['Linear', 'GAT', 'ICGAT']

mpc_config = {
    'ridge_coefficient': 0.0,
    'smoothness_coefficient': 0.0,
    'target_values_list': target['target_values'],
    'target_times_list': target['target_times'],
    'max_iter': 200,
    'loss_threshold': 1e-9,
    'opt_config': {'lr': 2e-0},
    'scheduler_config': {'patience': 5, 'factor': 0.5, 'min_lr': 1e-4}
}

fig, axes = plt.subplots(1, 1, figsize=(7, 4))
for model_name in model_names:
    loss_curve = []
    for i in range(101):
        true_loss_dict = pickle.load(open('bilevel_opt_result/optimal/true_loss/implicit_{}_7/{}_{}_{}.pkl'.format(model_name, num_x, num_heaters, i), 'rb'))
        x_trajectory_list = true_loss_dict['x_trajectory']
        u_trajectory_list = true_loss_dict['u_trajectory']
        log_trajectory_list = true_loss_dict['log_trajectory']
        loss_list = measure_design_opt_control(mpc_config, x_trajectory_list, u_trajectory_list)
        loss_curve.append(np.mean(loss_list))
    if model_name == 'Linear':
        # axes.plot(loss_curve, label='{}, True Cost'.format(model_name), color=color_dict[model_name])
        axes.plot(loss_curve, label='{}, Implicit'.format(model_name), color=color_dict[model_name])
    elif model_name == 'GAT':
        # axes.plot(loss_curve, label='{}NN, True Cost'.format(model_name[:-2]), color=color_dict[model_name])
        axes.plot(loss_curve, label='{}NN, Implicit'.format(model_name[:-2]), color=color_dict[model_name])
    else:
        # axes.plot(loss_curve, label='{}NN, True Cost'.format(model_name[:-2]), color=color_dict[model_name])
        axes.plot(1.0 * np.array(loss_curve), label='{}NN, Implicit (Ours)'.format(model_name[:-2]), color=color_dict[model_name])

# for model_name in model_names:
#     opt_result = pickle.load(open('bilevel_opt_result/implicit_{}/4_5.pkl'.format(model_name), 'rb'))
#     loss_trajectory = opt_result['opt_log']['total_loss_trajectory'][:, 8]
#     axes.plot(loss_trajectory/32, color=color_dict[model_name], linestyle='--', linewidth=3)
    # if model_name == 'Linear':
        # axes.plot(loss_trajectory/32, label='{}, Predicted Cost'.format(model_name), color=color_dict[model_name], linestyle='--', linewidth=3
    # elif model_name == 'GAT':
        # axes.plot(loss_trajectory/32, label='{}NN, Predicted Cost'.format(model_name[:-2]), color=color_dict[model_name], linestyle='--', linewidth=3)
    # else:
        # axes.plot(loss_trajectory/32, label='{}NN, Predicted Cost'.format(model_name[:-2]), color=color_dict[model_name], linestyle='--', linewidth=3)
    
    
label_size = 15

axes.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
# axes.set_yscale('log')
axes.ticklabel_format(axis='y', style='sci')
axes.set_ylim([0.0, 0.2])
# axes.legend(fancybox=True, shadow=True)
axes.grid(True, which='major', ls='--')
axes.set_xlabel('Optimization Steps', fontsize=label_size)
axes.set_ylabel('True Cost', fontsize=label_size)
# axes.legend(fontsize=label_size*0.9, loc=5)
plt.show()

In [ ]:
target = pickle.load(open('data/bilevel_design_opt/target.pkl', 'rb'))

color_dict = {
    'Linear': u'#1f77b4',
    'GAT': u'#ff7f0e',
    'ICGAT': u'#2ca02c'
}

def measure_design_opt_control(mpc_config, x_trajectory_list, u_trajectory_list):
    ridge_coefficient = mpc_config['ridge_coefficient']
    smoothness_coefficient = mpc_config['smoothness_coefficient']
    target_values_list = mpc_config['target_values_list']
    target_times_list = mpc_config['target_times_list']

    num_sensors = x_trajectory_list[0].shape[1]
    num_heaters = u_trajectory_list[0].shape[1]
    target_list = []
    for (target_values, target_times) in zip(target_values_list, target_times_list):
        target = np.array(generate_target_trajectory(target_values, target_times))
        target = np.reshape(target, newshape=(-1, 1))
        target = np.concatenate([target for _ in range(num_sensors)], axis=1)
        target_list.append(target)
    loss_list = []

    def compute_mpc_loss(x_traj, u_traj, target):
        loss_obj = np.square(x_traj[1:] - target).mean(axis=0).sum()
        loss_ridge = ridge_coefficient * np.square(u_traj).mean(axis=0).sum()
        u_prev = np.concatenate([np.zeros((1, num_heaters)), u_traj[:-1]], axis=0)
        loss_smoothness = smoothness_coefficient * np.square(u_traj - u_prev).mean(axis=0).sum()
        return loss_obj + loss_ridge + loss_smoothness

    for (x_traj, u_traj, target) in zip(x_trajectory_list, u_trajectory_list, target_list):
        loss_list.append(compute_mpc_loss(x_traj, u_traj, target))
    return np.array(loss_list)

num_x = 4
num_heaters = 5
model_names = ['Linear', 'GAT', 'ICGAT']

mpc_config = {
    'ridge_coefficient': 0.0,
    'smoothness_coefficient': 0.0,
    'target_values_list': target['target_values'],
    'target_times_list': target['target_times'],
    'max_iter': 200,
    'loss_threshold': 1e-9,
    'opt_config': {'lr': 2e-0},
    'scheduler_config': {'patience': 5, 'factor': 0.5, 'min_lr': 1e-4}
}

fig, axes = plt.subplots(1, 1, figsize=(7, 4))
for model_name in model_names:
    loss_curve = []
    for i in range(101):
        true_loss_dict = pickle.load(open('bilevel_opt_result/optimal/true_loss/implicit_{}_8/{}_{}_{}.pkl'.format(model_name, num_x, num_heaters, i), 'rb'))
        x_trajectory_list = true_loss_dict['x_trajectory']
        u_trajectory_list = true_loss_dict['u_trajectory']
        log_trajectory_list = true_loss_dict['log_trajectory']
        loss_list = measure_design_opt_control(mpc_config, x_trajectory_list, u_trajectory_list)
        loss_curve.append(np.mean(loss_list))
    if model_name == 'Linear':
        # axes.plot(loss_curve, label='{}, True Cost'.format(model_name), color=color_dict[model_name])
        axes.plot(loss_curve, label='{}, Implicit'.format(model_name), color=color_dict[model_name])
    elif model_name == 'GAT':
        # axes.plot(loss_curve, label='{}NN, True Cost'.format(model_name[:-2]), color=color_dict[model_name])
        axes.plot(loss_curve, label='{}NN, Implicit'.format(model_name[:-2]), color=color_dict[model_name])
    else:
        # axes.plot(loss_curve, label='{}NN, True Cost'.format(model_name[:-2]), color=color_dict[model_name])
        axes.plot(1.0 * np.array(loss_curve), label='{}NN, Implicit (Ours)'.format(model_name[:-2]), color=color_dict[model_name])

# for model_name in model_names:
#     opt_result = pickle.load(open('bilevel_opt_result/implicit_{}/4_5.pkl'.format(model_name), 'rb'))
#     loss_trajectory = opt_result['opt_log']['total_loss_trajectory'][:, 8]
#     axes.plot(loss_trajectory/32, color=color_dict[model_name], linestyle='--', linewidth=3)
    # if model_name == 'Linear':
        # axes.plot(loss_trajectory/32, label='{}, Predicted Cost'.format(model_name), color=color_dict[model_name], linestyle='--', linewidth=3
    # elif model_name == 'GAT':
        # axes.plot(loss_trajectory/32, label='{}NN, Predicted Cost'.format(model_name[:-2]), color=color_dict[model_name], linestyle='--', linewidth=3)
    # else:
        # axes.plot(loss_trajectory/32, label='{}NN, Predicted Cost'.format(model_name[:-2]), color=color_dict[model_name], linestyle='--', linewidth=3)
    
    
label_size = 15

axes.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
# axes.set_yscale('log')
axes.ticklabel_format(axis='y', style='sci')
axes.set_ylim([0.0, 0.2])
# axes.legend(fancybox=True, shadow=True)
axes.grid(True, which='major', ls='--')
axes.set_xlabel('Optimization Steps', fontsize=label_size)
axes.set_ylabel('True Cost', fontsize=label_size)
# axes.legend(fontsize=label_size*0.9, loc=5)
plt.show()

In [ ]:
target = pickle.load(open('data/bilevel_design_opt/target.pkl', 'rb'))

num_x = 4
num_heaters = 5
model_names = ['Linear', 'GAT', 'ICGAT']

mpc_config = {
    'ridge_coefficient': 0.0,
    'smoothness_coefficient': 0.0,
    'target_values_list': target['target_values'],
    'target_times_list': target['target_times'],
    'max_iter': 200,
    'loss_threshold': 1e-9,
    'opt_config': {'lr': 2e-0},
    'scheduler_config': {'patience': 5, 'factor': 0.5, 'min_lr': 1e-4}
}

fig, axes = plt.subplots(1, 1, figsize=(7, 4))

for model_name in model_names:
    opt_result = pickle.load(open('bilevel_opt_result/implicit_{}/4_5.pkl'.format(model_name), 'rb'))
    loss_trajectory = opt_result['opt_log']['total_loss_trajectory'][:, 8]
    if model_name == 'Linear':
        axes.plot(loss_trajectory/32, label='Linear, Implicit', color=color_dict[model_name], linewidth=3)
    elif model_name == 'ICGAT':
        axes.plot(loss_trajectory/32, label='ICGNN, Implicit (Ours)', color=color_dict[model_name], linewidth=3)
    else:
        axes.plot(loss_trajectory/32, label='GNN, Implicit', color=color_dict[model_name], linewidth=3)
    
    
label_size = 15

axes.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
# axes.set_yscale('log')
axes.ticklabel_format(axis='y', style='sci')
axes.set_ylim([0.0, 0.2])
axes.legend(fancybox=True, shadow=True)
axes.grid(True, which='major', ls='--')
axes.set_xlabel('Optimization Steps', fontsize=label_size)
axes.set_ylabel('Predicted Cost', fontsize=label_size)
axes.legend(fontsize=label_size*1.2, loc=1)
plt.show()

In [ ]:
target = pickle.load(open('data/bilevel_design_opt/target.pkl', 'rb'))

num_x = 3
num_heaters = 5
model_names = ['Linear', 'GAT', 'ICGAT']

mpc_config = {
    'ridge_coefficient': 0.0,
    'smoothness_coefficient': 0.0,
    'target_values_list': target['target_values'],
    'target_times_list': target['target_times'],
    'max_iter': 200,
    'loss_threshold': 1e-9,
    'opt_config': {'lr': 2e-0},
    'scheduler_config': {'patience': 5, 'factor': 0.5, 'min_lr': 1e-4}
}

fig, axes = plt.subplots(1, 1, figsize=(7, 4))

for model_name in model_names:
    opt_result = pickle.load(open('bilevel_opt_result/implicit_{}/4_5.pkl'.format(model_name), 'rb'))
    loss_trajectory = opt_result['opt_log']['total_loss_trajectory'][:, 7]
    if model_name == 'Linear':
        axes.plot(loss_trajectory/32, label='Linear, Implicit', color=color_dict[model_name], linewidth=3)
    elif model_name == 'ICGAT':
        axes.plot(loss_trajectory/32, label='ICGNN, Implicit (Ours)', color=color_dict[model_name], linewidth=3)
    else:
        axes.plot(loss_trajectory/32, label='GNN, Implicit', color=color_dict[model_name], linewidth=3)
    
    
label_size = 15

axes.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
# axes.set_yscale('log')
axes.ticklabel_format(axis='y', style='sci')
axes.set_ylim([0.0, 0.2])
axes.legend(fancybox=True, shadow=True)
axes.grid(True, which='major', ls='--')
axes.set_xlabel('Optimization Steps', fontsize=label_size)
axes.set_ylabel('Predicted Cost', fontsize=label_size)
axes.legend(fontsize=label_size*1.2, loc=1)
plt.show()

In [ ]:
problem = pickle.load(open('data/bilevel_design_opt/problem_4_5.pkl', 'rb'))
state_pos = problem['state_pos'][0]
model_names = ['Linear', 'GAT', 'ICGAT']
color_dict = {
    'Linear': u'#1f77b4',
    'GAT': u'#ff7f0e',
    'ICGAT': u'#2ca02c'
}
label_size = 16
tick_size = 11
plot_idxs = [0, 1, 10]
fig, axes = plt.subplots(len(model_names), len(plot_idxs)+1, figsize=(4*(len(plot_idxs)+1), 4.2*len(model_names)))
axes_flatten = axes.flatten()
for i, model_name in enumerate(model_names):
    opt_result = pickle.load(open('bilevel_opt_result/implicit_{}/4_5.pkl'.format(model_name), 'rb'))
    position_trajectory = opt_result['opt_log']['position_trajectory'][:, 8, :, :]
    optimal_idx = np.argmin(opt_result['opt_log']['total_loss_trajectory'][:, 8])
    model_plot_idxs = plot_idxs + [optimal_idx]
    for j in range(len(model_plot_idxs)):
        true_loss_dict = pickle.load(open('bilevel_opt_result/optimal/true_loss/implicit_{}_8/4_5_{}.pkl'.format(model_name, model_plot_idxs[j]), 'rb'))
        x_trajectory_list = true_loss_dict['x_trajectory']
        u_trajectory_list = true_loss_dict['u_trajectory']
        log_trajectory_list = true_loss_dict['log_trajectory']
        loss_list = measure_design_opt_control(mpc_config, x_trajectory_list, u_trajectory_list)
        axes_flatten[i * len(model_plot_idxs) + j].scatter(state_pos[:, 0], state_pos[:, 1], color='black', s=100)
        axes_flatten[i * len(model_plot_idxs) + j].scatter(position_trajectory[model_plot_idxs[j], :, 0], position_trajectory[model_plot_idxs[j], :, 1], color=color_dict[model_name], s=150, marker='s')
        if j != len(model_plot_idxs)-1:
            for k in range(position_trajectory.shape[-2]):
                current_x = position_trajectory[model_plot_idxs[j], k, 0]
                current_y = position_trajectory[model_plot_idxs[j], k, 1]
                dx = position_trajectory[model_plot_idxs[j]+1, k, 0] - position_trajectory[model_plot_idxs[j], k, 0]
                dy = position_trajectory[model_plot_idxs[j]+1, k, 1] - position_trajectory[model_plot_idxs[j], k, 1]
                axes_flatten[i * len(model_plot_idxs) + j].arrow(current_x, current_y, dx, dy, color=color_dict[model_name], width=0.03, head_length=0.04)
        axes_flatten[i * len(model_plot_idxs) + j].set_xlim(-1, 1)
        axes_flatten[i * len(model_plot_idxs) + j].set_ylim(-1, 1)
        axes_flatten[i * len(model_plot_idxs) + j].set_xticks([-1, -0.5, 0, 0.5, 1])
        axes_flatten[i * len(model_plot_idxs) + j].set_yticks([-1, -0.5, 0, 0.5, 1])
        axes_flatten[i * len(model_plot_idxs) + j].tick_params(axis='both', which='major', labelsize=tick_size)
        axes_flatten[i * len(model_plot_idxs) + j].set_xlabel('Cost: {}'.format(np.around(np.mean(loss_list), 4)), fontsize=label_size*1.2)
        if j == 0:
            axes_flatten[i * len(model_plot_idxs) + j].set_title('Initial Location', fontsize=label_size)
        elif j == 1:
            axes_flatten[i * len(model_plot_idxs) + j].set_title('1st Iteration', fontsize=label_size)
        elif j == len(model_plot_idxs)-1:
            axes_flatten[i * len(model_plot_idxs) + j].set_title('Optimal Location', fontsize=label_size)
        else:
            axes_flatten[i * len(model_plot_idxs) + j].set_title('{}th Iteration'.format(plot_idxs[j]), fontsize=label_size)
fig.tight_layout()

# Figure 3. Heater Location Optimization Performance: Loss vs # of heaters

## Preivous Option

In [ ]:
num_x_list = [3, 4, 5]
num_heaters_list = [5, 10, 15, 20]

fig, axes = plt.subplots(1, len(num_x_list), figsize=(4.5 * len(num_x_list), 3.5 * 1))
axes_flatten = axes.flatten()
color_dict = {
    'Linear': u'#1f77b4',
    'GAT': u'#ff7f0e',
    'ICGAT': u'#2ca02c',
    'cma_es': u'#d62728',
    'single_layer': u'#9467bd',
    'implicit': u'#2ca02c',
}
model_names = ['Linear', 'GAT', 'ICGAT']
solver_names = ['single_layer', 'cma_es', 'implicit']
label_size = 15
tick_size = 10
legend_size = 10
for i, num_x in enumerate(num_x_list):
    for model_name in model_names:
        loss_list = []
        for num_heaters in num_heaters_list:
            optimal_result = pickle.load(open('bilevel_opt_result/optimal/implicit_{}/optimal_result_{}_{}.pkl'.format(model_name, num_x, num_heaters), 'rb'))
            x_trajectory_list = optimal_result['x_trajectory_list']
            u_trajectory_list = optimal_result['u_trajectory_list']
            log_trajectory_list = optimal_result['log_trajectory_list']
            loss = measure_design_opt_control(mpc_config, x_trajectory_list, u_trajectory_list)
            loss_list.append(np.mean(loss))
        if model_name == 'Linear':
            label_name ='{}, Implicit'.format(model_name)
        elif model_name == 'GAT':
            label_name = '{}NN, Implicit'.format(model_name[:-2])
        else:
            label_name = '{}NN, Implicit (Ours)'.format(model_name[:-2])
        axes_flatten[i].plot(num_heaters_list, loss_list, color=color_dict[model_name], linewidth=3, label=label_name)
        axes_flatten[i].set_title('Number of Sensors: {}'.format(num_x**2), fontsize=label_size)
        axes_flatten[i].set_ylabel('True Cost', fontsize=label_size * 0.8)
        axes_flatten[i].set_xlabel('Number of Heaters', fontsize=label_size * 0.8)
        axes_flatten[i].set_xticks(num_heaters_list)
        # axes_flatten[i].ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
        axes_flatten[i].tick_params(axis='both', which='major', labelsize=tick_size)
        axes_flatten[i].set_yscale('log')

axes_flatten[i].legend(fancybox=True, shadow=True, fontsize=legend_size)
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, len(num_x_list), figsize=(4 * len(num_x_list), 3 * 1))
axes_flatten = axes.flatten()
label_size = 15
tick_size = 12
legend_size = 10

for i, num_x in enumerate(num_x_list):
    for solver_name in solver_names:
        loss_list = []
        for num_heaters in num_heaters_list:
            optimal_result = pickle.load(open('bilevel_opt_result/optimal/{}_ICGAT/optimal_result_{}_{}.pkl'.format(solver_name, num_x, num_heaters), 'rb'))
            x_trajectory_list = optimal_result['x_trajectory_list']
            u_trajectory_list = optimal_result['u_trajectory_list']
            log_trajectory_list = optimal_result['log_trajectory_list']
            loss = measure_design_opt_control(mpc_config, x_trajectory_list, u_trajectory_list)
            loss_list.append(np.mean(loss))
        label_name = 'ICGNN, {}'.format(solver_name)
        if solver_name == 'implicit':
            label_name = 'ICGNN, {} (Ours)'.format(solver_name)
        elif solver_name == 'cma_es':
            label_name = 'ICGNN, Nested'
        else:
            label_name = 'ICGNN, Single-Layer'
        axes_flatten[i].plot(num_heaters_list, loss_list, color=color_dict[solver_name], linewidth=3, label=label_name)
        axes_flatten[i].set_title('Number of Sensors: {}'.format(num_x**2), fontsize=label_size)
        axes_flatten[i].set_ylabel('Control Loss', fontsize=label_size)
        axes_flatten[i].set_xlabel('Number of Heaters', fontsize=label_size)
        axes_flatten[i].set_xticks(num_heaters_list)
        axes_flatten[i].tick_params(axis='both', which='major', labelsize=tick_size)
        axes_flatten[i].set_yscale('log')
        
axes_flatten[-1].legend(fancybox=True, shadow=True, fontsize=legend_size)
fig.tight_layout()

## New Option

In [ ]:
num_x_list = [3, 4, 5]
num_heaters_list = [5, 10, 15, 20]
num_repeats = 10

fig, axes = plt.subplots(1, len(num_x_list), figsize=(4.5 * len(num_x_list), 3.5 * 1))
axes_flatten = axes.flatten()
color_dict = {
    'Linear': u'#1f77b4',
    'GAT': u'#ff7f0e',
    'ICGAT': u'#2ca02c',
    'cma_es': u'#d62728',
    'single_layer': u'#9467bd',
    'implicit': u'#2ca02c',
}
model_names = ['Linear', 'GAT', 'ICGAT']
solver_names = ['implicit']
label_size = 15
tick_size = 10
legend_size = 10
for i, num_x in enumerate(num_x_list):
    for model_name in model_names:
        loss_list_model_mean = []
        loss_list_model_std = []
        for num_heaters in num_heaters_list:
            loss_list_repeats = []
            for j in range(num_repeats):
                optimal_result = pickle.load(open('bilevel_opt_result/optimal/implicit_{}/{}_{}_{}.pkl'.format(model_name, num_x, num_heaters, j), 'rb'))
                x_trajectory_list = optimal_result['x_trajectory_list']
                u_trajectory_list = optimal_result['u_trajectory_list']
                log_trajectory_list = optimal_result['log_trajectory_list']
                loss = measure_design_opt_control(mpc_config, x_trajectory_list, u_trajectory_list)
                loss_list_repeats.append(np.mean(loss))
            loss_list_model_mean.append(np.mean(loss_list_repeats))
            loss_list_model_std.append(np.std(loss_list_repeats))
        if model_name == 'Linear':
            label_name ='{}, Implicit'.format(model_name)
        elif model_name == 'GAT':
            label_name = '{}NN, Implicit'.format(model_name[:-2])
        else:
            label_name = '{}NN, Implicit (Ours)'.format(model_name[:-2])
        loss_list_model_mean = np.array(loss_list_model_mean)
        loss_list_model_std = np.array(loss_list_model_std)
        axes_flatten[i].plot(num_heaters_list, loss_list_model_mean, color=color_dict[model_name], linewidth=3, label=label_name)
        axes_flatten[i].fill_between(num_heaters_list, loss_list_model_mean - 0.5 * loss_list_model_std, loss_list_model_mean + 0.5 * loss_list_model_std, color=color_dict[model_name], alpha=0.3)
        axes_flatten[i].set_title('Number of Sensors: {}'.format(num_x**2), fontsize=label_size)
        axes_flatten[i].set_ylabel('True Cost', fontsize=label_size * 0.8)
        axes_flatten[i].set_xlabel('Number of Heaters', fontsize=label_size * 0.8)
        axes_flatten[i].set_xticks(num_heaters_list)
        # axes_flatten[i].ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
        axes_flatten[i].tick_params(axis='both', which='major', labelsize=tick_size)
        axes_flatten[i].set_yscale('log')

axes_flatten[i].legend(fancybox=True, shadow=True, fontsize=legend_size)
fig.tight_layout()
plt.show()

In [ ]:
num_x_list = [3, 4, 5]
num_heaters_list = [5, 10, 15, 20]
num_repeats = 10

fig, axes = plt.subplots(1, len(num_x_list), figsize=(4.5 * len(num_x_list), 3.5 * 1))
axes_flatten = axes.flatten()
color_dict = {
    'Linear': u'#1f77b4',
    'GAT': u'#ff7f0e',
    'ICGAT': u'#2ca02c',
    'cma_es': u'#d62728',
    'single_layer': u'#9467bd',
    'implicit': u'#2ca02c',
}
model_names = ['ICGAT']
solver_names = ['cma_es', 'single_layer', 'implicit']
label_size = 15
tick_size = 10
legend_size = 10
for i, num_x in enumerate(num_x_list):
    for solver_name in solver_names:
        loss_list_model_mean = []
        loss_list_model_std = []
        for num_heaters in num_heaters_list:
            loss_list_repeats = []
            for j in range(num_repeats):
                optimal_result = pickle.load(open('bilevel_opt_result/optimal/{}_ICGAT/{}_{}_{}.pkl'.format(solver_name, num_x, num_heaters, j), 'rb'))
                x_trajectory_list = optimal_result['x_trajectory_list']
                u_trajectory_list = optimal_result['u_trajectory_list']
                log_trajectory_list = optimal_result['log_trajectory_list']
                loss = measure_design_opt_control(mpc_config, x_trajectory_list, u_trajectory_list)
                loss_list_repeats.append(np.mean(loss))
            loss_list_model_mean.append(np.mean(loss_list_repeats))
            loss_list_model_std.append(np.std(loss_list_repeats))
        if solver_name == 'cma_es':
            label_name ='ICGNN, CMA-ES'
        elif solver_name == 'single_layer':
            label_name = 'ICGNN, Single-Layer'
        else:
            label_name = 'ICGNN, Implicit (Ours)'
        loss_list_model_mean = np.array(loss_list_model_mean)
        loss_list_model_std = np.array(loss_list_model_std)
        axes_flatten[i].plot(num_heaters_list, loss_list_model_mean, color=color_dict[solver_name], linewidth=3, label=label_name)
        axes_flatten[i].fill_between(num_heaters_list, loss_list_model_mean - 0.5 * loss_list_model_std, loss_list_model_mean + 0.5 * loss_list_model_std, color=color_dict[solver_name], alpha=0.3)
        axes_flatten[i].set_title('Number of Sensors: {}'.format(num_x**2), fontsize=label_size)
        axes_flatten[i].set_ylabel('True Cost', fontsize=label_size * 0.8)
        axes_flatten[i].set_xlabel('Number of Heaters', fontsize=label_size * 0.8)
        axes_flatten[i].set_xticks(num_heaters_list)
        # axes_flatten[i].ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
        axes_flatten[i].tick_params(axis='both', which='major', labelsize=tick_size)
        axes_flatten[i].set_yscale('log')

axes_flatten[i].legend(fancybox=True, shadow=True, fontsize=legend_size)
fig.tight_layout()
plt.show()